### An Example of a DNNClassifier for the Iris dataset.

In [1]:
import tensorflow as tf

In [2]:
import iris_data

In [3]:
args = {"batch_size": 100, "train_steps": 1000}

In [4]:
# Fetch the data
(train_x, train_y), (test_x, test_y) = iris_data.load_data()

In [5]:
print(type(train_x))
train_x.head()

<class 'pandas.core.frame.DataFrame'>


,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [6]:
print(type(train_y))
train_y.head()

<class 'pandas.core.series.Series'>


0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [7]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [8]:
my_feature_columns

[_NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [9]:
# Build 2 hidden layer DNN with 10, 10 units respectively.
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_secs = 1,  # Save checkpoints every 20 minutes.
    keep_checkpoint_max = 5,       # Retain the 10 most recent checkpoints.
)
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3,
    # Model checkpoint dir
    model_dir='models/iris',
    # Checkpoint config
    config=my_checkpointing_config)

INFO:tensorflow:Using config: {'_model_dir': 'models/iris', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 1, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fac61eafc18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
# Train the Model.
classifier.train(
    input_fn=lambda:iris_data.train_input_fn(train_x, train_y,
                                             args["batch_size"]),
    steps=args["train_steps"])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/iris/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1001 into models/iris/model.ckpt.
INFO:tensorflow:loss = 4.135279, step = 1000
INFO:tensorflow:global_step/sec: 447.821
INFO:tensorflow:loss = 5.435881, step = 1100 (0.224 sec)
INFO:tensorflow:global_step/sec: 539.828
INFO:tensorflow:loss = 4.734592, step = 1200 (0.185 sec)
INFO:tensorflow:global_step/sec: 526.951
INFO:tensorflow:loss = 5.1261177, step = 1300 (0.190 sec)
INFO:tensorflow:global_step/sec: 508.75
INFO:tensorflow:loss = 2.4982183, step = 1400 (0.197 sec)
INFO:tensorflow:Saving checkpoints for 1445 into models/iris/model.ckpt.
INFO:tensorflow:global_step/sec: 393.703
INFO:tensorflow:loss = 5.4822683, step = 1500 (0.254 sec)
INFO:tenso

In [11]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:iris_data.eval_input_fn(test_x, test_y,
                                            args["batch_size"]))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-03-14:23:36
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/iris/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-03-14:23:36
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.96666664, average_loss = 0.06902214, global_step = 2000, loss = 2.0706642


In [12]:
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy: 0.967



In [13]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
    input_fn=lambda:iris_data.eval_input_fn(predict_x,
                                            labels=None,
                                            batch_size=args["batch_size"]))

for pred_dict, expec in zip(predictions, expected):
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(iris_data.SPECIES[class_id],
                          100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/iris/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (99.9%), expected "Setosa"

Prediction is "Versicolor" (99.9%), expected "Versicolor"

Prediction is "Virginica" (99.0%), expected "Virginica"
